In [1]:
import numpy as np
import casadi as ca
vertices = np.array([[0, 0], [1, 0], [1, 1], [0, 1]])
distance_to = [(0.5, 0.5)]
a = vertices
b = np.roll(a, -1, axis=0)
edge = b - a
v = distance_to[:2] - a
pq = (
    v
    - edge
    * np.clip(np.sum(v * edge, axis=1) / np.sum(edge * edge, axis=1), 0, 1)[
        :, None
    ]
)
distance = np.min(np.sum(pq**2, axis=1))

v2 = distance_to[:2] - b
val3 = (np.roll(edge, 1, axis=1) * v)
val3 = val3[:, 1] - val3[:, 0]
condition = np.stack([v[:, 1] >= 0, v2[:, 1] < 0, val3 > 0])
not_condition = np.stack([v[:, 1] < 0, v2[:, 1] >= 0, val3 < 0])
print(condition.shape, not_condition.shape)
condition = np.all(condition, axis=0)
not_condition = np.all(np.all(not_condition, axis=0))
condition = np.all(np.sum(condition))
s = -1 if condition or not_condition else 1
print(np.sqrt(distance) * s)

(3, 4) (3, 4)
-0.5


In [2]:
vertices = np.array([[0, 0], [1, 0], [1, 1], [0, 1]])
distance_to = ca.SX.sym("distance_to", 2)
a = vertices
b = np.roll(a, -1, axis=0)
edge = b - a
v = ca.repmat(distance_to[:2].T, a.shape[0], 1) - a
# pq = (
#     v
#     - edge
#     * np.clip(np.sum(v * edge, axis=1) / np.sum(edge * edge, axis=1), 0, 1)[
#         :, None
#     ]
# )
# distance = np.min(np.sum(pq**2, axis=1))

# v2 = distance_to[:2] - b
# val3 = (np.roll(edge, 1, axis=1) * v)
# val3 = val3[:, 1] - val3[:, 0]
# condition = np.stack([v[:, 1] >= 0, v2[:, 1] < 0, val3 > 0])
# not_condition = np.stack([v[:, 1] < 0, v2[:, 1] >= 0, val3 < 0])
# condition = np.all(condition, axis=0)
# not_condition = np.all(np.all(not_condition, axis=0))
# condition = np.all(np.sum(condition))
# s = -1 if condition or not_condition else 1
# print(np.sqrt(distance) * s)

In [13]:

v = ca.repmat(distance_to[:2].T, a.shape[0], 1) - a
pq = v - edge * ca.fmin(ca.fmax(ca.sum2(v * edge) / ca.sum2(edge * edge), 0), 1)
distance = ca.mmin(ca.sum2(pq**2))

v2 = ca.repmat(distance_to[:2].T, b.shape[0], 1) - b
val3 = np.roll(edge, 1, axis=1) * v
val3 = val3[:, 1] - val3[:, 0]
condition = ca.horzcat(v[:, 1] >= 0, v2[:, 1] < 0, val3 > 0)
not_condition = ca.horzcat(v[:, 1] < 0, v2[:, 1] >= 0, val3 < 0)
condition = ca.sum2(ca.sum1(condition))
not_condition = ca.sum2(ca.sum1(not_condition))
print(ca.if_else(
    ca.sum1(ca.vertcat(condition, not_condition)),
    ca.sqrt(distance) * -1,
    ca.sqrt(distance) * 1,
))

@1=0, @2=1, @3=(distance_to_1-@2), @4=(distance_to_1-@2), @5=(distance_to_1-@2), @6=(distance_to_1-@2), @7=(distance_to_0-@2), @8=(-@7), @9=(-@3), @10=(((((((@1<=distance_to_1)+(@1<=distance_to_1))+(@1<=@3))+(@1<=@4))+((((distance_to_1<@1)+(@5<@1))+(@6<@1))+(distance_to_1<@1)))+((((@1<distance_to_1)+(@1<@8))+(@1<@9))+(@1<distance_to_0)))+((((((distance_to_1<@1)+(distance_to_1<@1))+(@3<@1))+(@4<@1))+((((@1<=distance_to_1)+(@1<=@5))+(@1<=@6))+(@1<=distance_to_1)))+((((distance_to_1<@1)+(@8<@1))+(@9<@1))+(distance_to_0<@1)))), @11=(distance_to_0-@2), @12=fmin(fmin(fmin((sq((distance_to_0-fmin(fmax(distance_to_0,@1),@2)))+sq(distance_to_1)),(sq(@7)+sq((distance_to_1-fmin(fmax(distance_to_1,@1),@2))))),(sq((@11+fmin(fmax((-@11),@1),@2)))+sq(@3))),(sq(distance_to_0)+sq((@4+fmin(fmax((-@4),@1),@2))))), ((@10?(-sqrt(@12)):0)+((!@10)?sqrt(@12):0))
